# Database overview notebook

> First time use: follow instructions in the README.md file in this directory.

## Setup

In [1]:
from timelink.mhk.utilities import get_engine, get_dbnames, get_connection_string
from sqlalchemy import engine, text, MetaData
import platform

machine = platform.node()

print(f'Available databases at {machine}:')
print(get_dbnames())
#%load_ext sql
#%config SqlMagic.displaycon=False

Available databases at imac-jrc.local:
['china_coimbra', 'soure_ipm', 'ucprosop']


## Connect to database

### Choose the database

In [3]:
if machine == 'joaquims-mbpr.local':
    db = 'soure_editor'
elif machine == 'imac-jrc.local':
    db = 'soure_ipm'

connection_string = get_connection_string(db)
engine = get_engine(db)

## Database status

In [4]:
with engine.connect() as connection:
    classes = connection.execute(
        text('select class,count(*) as n from entities group by class')
    )
    classes
for c in classes:
    print(c['class'],c.n)

act 10844
acta 97
acusacoes 315
aforamento 2
aregister 5
attribute 65441
cartaperdao 2
caso 40
class 27
crime 1
divida 3
escritura 243
geoentity 9
good 6
group_element 10870
item 92
object 2
person 51438
relation 86656
rperson 1051
source 184


## Map MHK database

In [6]:

from sqlalchemy import Column,String,Integer,DateTime, ForeignKey
from sqlalchemy.orm import declarative_base, relationship

Base = declarative_base()


## Entities
     
        |-----------|--------------|------|-----|---------------------+-----------------------------+
        | Field     | Type         | Null | Key | Default             | Extra                       |
        +-----------+--------------+------+-----+---------------------+-----------------------------+
        | id        | varchar(64)  | NO   | PRI | NULL                |                             |
        | class     | varchar(64)  | YES  | MUL | NULL                |                             |
        | inside    | varchar(64)  | YES  | MUL | NULL                |                             |
        | the_order | decimal(6,0) | YES  |     | NULL                |                             |
        | the_level | decimal(6,0) | YES  |     | NULL                |                             |
        | the_line  | decimal(6,0) | YES  |     | NULL                |                             |
        | groupname | varchar(32)  | YES  | MUL | NULL                |                             |
        | updated   | timestamp    | NO   | MUL | CURRENT_TIMESTAMP   | on update CURRENT_TIMESTAMP |
        | indexed   | timestamp    | NO   | MUL | 1974-04-25 00:00:01 |                             |
        +-----------+--------------+------+-----+---------------------+-----------------------------+

In [7]:
from sqlalchemy import case

class Entity(Base):
    __tablename__ = 'entities'

    id = Column(String, primary_key=True)
    pom_class = Column('class',String,ForeignKey('classes.id'))
    inside = Column(String, ForeignKey('entities.id'))
    the_order = Column(Integer)
    the_level = Column(Integer)
    the_line = Column(Integer)
    groupname = Column(String)
    updated = Column(DateTime)
    indexed = Column(DateTime)


    rels_in = relationship("Relation", back_populates="dest")
    rels_out = relationship("Relation",back_populates="org")

    # see https://docs.sqlalchemy.org/en/14/orm/inheritance.html
    # To handle non mapped pom_class see https://github.com/sqlalchemy/sqlalchemy/issues/5445
    #
    #    __mapper_args__ = {
    #       "polymorphic_identity": "entity",
    #    "polymorphic_on": case(
    #        [(type.in_(["parent", "child"]), type)], else_="entity"
    #    ),
    # 
    #  This defines what mappings do exist
    # [aclass.__mapper_args__['polymorphic_identity'] for aclass in Entity.__subclasses__()]

    __mapper_args__ = {
        'polymorphic_identity': 'entity',
        'polymorphic_on':case(
            [(pom_class.in_(['attribute', 'relation', 'person', 'object', 'act', 'source', 'class']), pom_class)], 
                else_="entity")
    }

    # untested
    @classmethod  # untested
    def get_subclasses(cls):
        for subclass in cls.__subclasses__():
            yield from subclass.get_subclasses()
            yield subclass

    @classmethod 
    def mapped_pom_classes(cls):
        return [aclass.__mapper_args__['polymorphic_identity'] 
                    for aclass 
                    in Entity.__subclasses__()]

    @classmethod
    def table_to_orm(cls):
        """
        Return a dict with table name as key and ORM class as value
        """
        return {subclass.__mapper__.local_table.name:subclass for subclass in cls.get_subclasses()}


    @classmethod
    def pom_class_to_orm(cls):
        """
        Return a dict with pom_class as key and ORM class as valu
        """
        return {subclass.__mapper__.polymorphic_identity:subclass for subclass in cls.get_subclasses()}


    @classmethod
    def get_orm_for_table(cls,table: String):
        """
        Entity.get_orm_for_table("acts")

        will return the ORM class handling the "acts" table
        """
        return cls.table_to_orm().get(table,None)

    @classmethod
    def get_orm_for_pom_class(cls,pom_class: String):
        """
        Entity.get_orgm_for_pom_class("act")

        will return the ORM class corresponding to the pom_class "act" 
        """
        return cls.pom_class_to_orm().get(pom_class,None)
    

    def __repr__(self):
        return (
            f'Entity(id="{self.id}", '
            f'pom_class="{self.pom_class}",'
            f'inside="{self.inside}", '
            f'the_order={self.the_order}, '
            f'the_level={self.the_level}, '
            f'the_line={self.the_line}, '
            f'groupname="{self.groupname}", '
            
            f'updated={self.updated}, '
            f'indexed={self.indexed},'
            f')'
        )
        
    def __str__(self):
        return (f'{self.groupname}${self.id}/type={self.pom_class}')


## Attributes
    +-----------+----------------+------+-----+---------+-------+
    | Field     | Type           | Null | Key | Default | Extra |
    +-----------+----------------+------+-----+---------+-------+
    | id        | varchar(64)    | NO   | PRI | NULL    |       |
    | entity    | varchar(64)    | YES  | MUL | NULL    |       |
    | the_type  | varchar(512)   | YES  | MUL | NULL    |       |
    | the_value | varchar(1024)  | YES  | MUL | NULL    |       |
    | the_date  | varchar(24)    | YES  | MUL | NULL    |       |
    | obs       | varchar(16000) | YES  |     | NULL    |       |
    +-----------+----------------+------+-----+---------+-------+

In [8]:

class Attribute(Entity):
    __tablename__ = 'attributes'

    id = Column(String, ForeignKey('entities.id'), primary_key=True)
    entity = Column(String,ForeignKey('entities.id'))
    the_type = Column(String)
    the_value = Column(String)
    the_date = Column(String)
    obs = Column(String)

    the_entity = relationship("Entity",foreign_keys=[entity],back_populates="attributes",)

    __mapper_args__ = {
        'polymorphic_identity':'attribute',     
        'inherit_condition': id == Entity.id
    }

    def __repr__(self):
        sr = super().__repr__()
        return (
            f'Attribute(id={sr}, '
            f'entity="{self.entity}", '
            f'the_type="{self.the_type}", '
            f'the_value="{self.the_value}", '
            f'the_date="{self.the_date}"", '
            f'obs={self.obs}'
            f')'
        )

    def __str__(self):
        r = f'     ls${self.the_type}/{self.the_value}/{self.the_date}'
        if self.obs is not None:
                r = (f'{r}/obs={self.obs}')
        return r


Entity.attributes = relationship("Attribute", foreign_keys=[Attribute.entity], back_populates="the_entity")


## Relations   
    +-------------+----------------+------+-----+---------+-------+
    | Field       | Type           | Null | Key | Default | Extra |
    +-------------+----------------+------+-----+---------+-------+
    | id          | varchar(64)    | NO   | PRI | NULL    |       |
    | origin      | varchar(64)    | YES  | MUL | NULL    |       |
    | destination | varchar(64)    | YES  | MUL | NULL    |       |
    | the_date    | varchar(24)    | YES  | MUL | NULL    |       |
    | the_type    | varchar(32)    | YES  | MUL | NULL    |       |
    | the_value   | varchar(256)   | YES  | MUL | NULL    |       |
    | obs         | varchar(16000) | YES  |     | NULL    |       |
    +-------------+----------------+------+-----+---------+-------+

In [9]:
class Relation(Entity):  # should extend Entity but gives error

    __tablename__ = 'relations'

    id = Column(String,ForeignKey('entities.id'), primary_key=True)
    #rel_entity = relationship("Entity",foreign_keys='id',back_populates='rel')
    origin = Column(String,ForeignKey('entities.id'))
    org = relationship(Entity,foreign_keys=[origin], back_populates='rels_out')

    destination = Column(String,ForeignKey('entities.id'))
    dest = relationship("Entity",foreign_keys=[destination], back_populates="rels_in")
    the_type = Column(String)
    the_value = Column(String)
    the_date = Column(String)
    obs = Column(String)

    __mapper_args__ = {
        'polymorphic_identity':'relation',
        'inherit_condition': id == Entity.id
    }

    def __repr__(self):
        sr = super().__repr__()
        return (
            f'Relation(id={sr}, '
            f'origin="{self.origin}", '
            f'destination="{self.destination}", '
            f'the_type="{self.the_type}", '
            f'the_value="{self.the_value}", '
            f'the_date="{self.the_date}"", '
            f'obs={self.obs}'
            f')'
        )

    def __str__(self):
        if self.dest is not None and self.dest.pom_class == 'person':
            r = f'rel${self.the_type}/{self.the_value}/{self.dest.name}/{self.destination}/{self.the_date}'
        else:
            r = f'rel${self.the_type}/{self.the_value}/{self.destination}/{self.the_date}'
        if self.obs is not None:
                r = (f'{r}  /obs={self.obs}')
        return r

Entity.rels_out = relationship("Relation",foreign_keys=[Relation.origin],back_populates="dest")
Entity.rels_in = relationship("Relation",foreign_keys=[Relation.destination],back_populates="org")

/var/folders/6m/_0n598297cj_bc2ql0rynnx00000gp/T/ipykernel_89793/3788928658.py:44: SAWarning: Property Entity.rels_out on mapped class Entity->entities being replaced with new property Entity.rels_out; the old property will be discarded
  Entity.rels_out = relationship("Relation",foreign_keys=[Relation.origin],back_populates="dest")
/var/folders/6m/_0n598297cj_bc2ql0rynnx00000gp/T/ipykernel_89793/3788928658.py:44: SAWarning: Property Entity.rels_out on mapped class Attribute->attributes being replaced with new property Entity.rels_out; the old property will be discarded
  Entity.rels_out = relationship("Relation",foreign_keys=[Relation.origin],back_populates="dest")
/var/folders/6m/_0n598297cj_bc2ql0rynnx00000gp/T/ipykernel_89793/3788928658.py:44: SAWarning: Property Entity.rels_out on mapped class Relation->relations being replaced with new property Entity.rels_out; the old property will be discarded
  Entity.rels_out = relationship("Relation",foreign_keys=[Relation.origin],back_popul

## Persons    
    +-------+----------------+------+-----+---------+-------+
    | Field | Type           | Null | Key | Default | Extra |
    +-------+----------------+------+-----+---------+-------+
    | id    | varchar(64)    | NO   | PRI | NULL    |       |
    | name  | varchar(128)   | YES  | MUL | NULL    |       |
    | sex   | char(1)        | YES  |     | NULL    |       |
    | obs   | varchar(16000) | YES  |     | NULL    |       |
    +-------+----------------+------+-----+---------+-------+

In [10]:
class Person(Entity):
    __tablename__ = 'persons'

    id = Column(String, ForeignKey('entities.id'), primary_key=True)
    name = Column(String)
    sex = Column(String(1))
    obs = Column(String)

    __mapper_args__ = {
        'polymorphic_identity':'person'
    }

    def __repr__(self):
        sr = super().__repr__()
        return (
            f'Person(id={sr}, '
            f'name="{self.name}", '
            f'sex="{self.sex}", '
            f'obs={self.obs}'
            f')'
        )

        
    def __str__(self):
        r = f'{self.groupname}${self.name}/{self.sex}/id={self.id}'
        if self.obs is not None:
                r = (f'{r}  /obs={self.obs}')
        return r

## Objects

    +----------+----------------+------+-----+---------+-------+
    | Field    | Type           | Null | Key | Default | Extra |
    +----------+----------------+------+-----+---------+-------+
    | id       | varchar(64)    | NO   | PRI | NULL    |       |
    | name     | varchar(64)    | YES  | MUL | NULL    |       |
    | the_type | varchar(32)    | YES  |     | NULL    |       |
    | obs      | varchar(16000) | YES  |     | NULL    |       |
    +----------+----------------+------+-----+---------+-------+

In [11]:
class Object(Entity):
    __tablename__ = 'objects'

    id = Column(String, ForeignKey('entities.id'), primary_key=True)
    name = Column(String)
    the_type = Column(String(32))
    obs = Column(String)

    __mapper_args__ = {
        'polymorphic_identity':'object'
    }

    def __repr__(self):
        sr = super().__repr__()
        return(
            f'Object(id={sr}, '
            f'name="{self.name}", '
            f'the_type="{self.the_type}", '
            f'obs={self.obs}'
            f')'
        )

    def __str__(self):
        r = f'{self.groupname}${self.name}/{self.the_type}/id={self.id}'
        if self.obs is not None:
                r = (f'{r}  /obs={self.obs}')
        return r


## Acts

    +----------+----------------+------+-----+---------+-------+
    | Field    | Type           | Null | Key | Default | Extra |
    +----------+----------------+------+-----+---------+-------+
    | id       | varchar(64)    | NO   | PRI | NULL    |       |
    | the_type | varchar(32)    | YES  |     | NULL    |       |
    | the_date | varchar(24)    | YES  | MUL | NULL    |       |
    | loc      | varchar(64)    | YES  |     | NULL    |       |
    | ref      | varchar(64)    | YES  |     | NULL    |       |
    | obs      | varchar(16000) | YES  |     | NULL    |       |
    +----------+----------------+------+-----+---------+-------+


In [12]:
class Act(Entity):
    __tablename__ = 'acts'

    id = Column(String, ForeignKey('entities.id'), primary_key=True)
    the_type = Column(String(32))
    the_date = Column(String)
    loc = Column(String)
    ref = Column(String)
    obs = Column(String)

    __mapper_args__ = {
        'polymorphic_identity':'act'
    }


    def __repr__(self):
        sr = super().__repr__()
        return (
            f'Act(id={sr}, '
            f'the_type="{self.the_type}", '
            f'the_date="{self.the_date}", '
            f'local="{self.loc}", '
            f'ref="{self.ref}", '
            f'obs={self.obs}'
            f')'
            )

    def __str__(self):
        r = f'{self.groupname}${self.id}/{self.the_date}/type={self.the_type}/ref={self.ref}/loc={self.loc}'
        if self.obs is not None:
                r = (f'{r}  /obs={self.obs}')
        return r


## Sources

    +-----------+----------------+------+-----+---------+-------+
    | Field     | Type           | Null | Key | Default | Extra |
    +-----------+----------------+------+-----+---------+-------+
    | id        | varchar(64)    | NO   | PRI | NULL    |       |
    | the_type  | varchar(32)    | YES  |     | NULL    |       |
    | the_date  | varchar(24)    | YES  |     | NULL    |       |
    | loc       | varchar(64)    | YES  |     | NULL    |       |
    | ref       | varchar(64)    | YES  |     | NULL    |       |
    | kleiofile | varchar(1024)  | YES  |     | NULL    |       |
    | replaces  | varchar(254)   | YES  |     | NULL    |       |
    | obs       | varchar(16000) | YES  |     | NULL    |       |
    +-----------+----------------+------+-----+---------+-------+

In [13]:
class Source(Entity):
    __tablename__ = 'sources'

    id = Column(String, ForeignKey('entities.id'), primary_key=True)
    the_type = Column(String(32))
    the_date = Column(String)
    loc = Column(String)
    ref = Column(String)
    kleiofile = Column(String)
    replaces = Column(String)
    obs = Column(String)

    __mapper_args__ = {
        'polymorphic_identity':'source'
    }

    def __repr__(self):
        sr = super().__repr__()
        return (
            f'Source(id={sr}, '
            f'the_type="{self.the_type}", '
            f'the_date="{self.the_date}", '
            f'local="{self.loc}", '
            f'ref="{self.ref}", '
            f'kleiofile="{self.kleiofile}", '
            f'replaces="{self.replaces}", '
            f'obs={self.obs}'
            f')'
        )

    def __str__(self):
        r = f'{self.groupname}${self.id}/{self.the_date}/type={self.the_type}/ref={self.ref}/loc={self.loc}/kleiofile={self.kleiofile}/replaces={self.replaces}'
        if self.obs is not None:
                r = (f'{r}  /obs={self.obs}')
        return r

## POM Classes

    +------------+-------------+------+-----+---------+-------+
    | Field      | Type        | Null | Key | Default | Extra |
    +------------+-------------+------+-----+---------+-------+
    | id         | varchar(64) | NO   | PRI | NULL    |       |
    | table_name | varchar(32) | YES  |     | NULL    |       |
    | group_name | varchar(32) | YES  |     | NULL    |       |
    | super      | varchar(64) | YES  |     | NULL    |       |
    +------------+-------------+------+-----+---------+-------+



In [14]:
from typing import Optional

class PomSomMapper(Entity):
    __tablename__ = 'classes'

    id = Column(String, ForeignKey('entities.id'), primary_key=True)
    table_name = Column(String)
    class_group = Column("group_name",String(32))
    super_class = Column("super", String)

    __mapper_args__ = {
        'polymorphic_identity':'class',
        'inherit_condition': id == Entity.id
    }

    def ensure_mapping(self,session):
        """
        Ensure that a table exists to support this class.

        Checks in the metadata object if there is a table definition for this table.
        NOT TESTED
        """
        metadata_obj = type(self).metadata
        pytables = metadata_obj.tables
        if self.table_name in pytables.keys():
            return pytables[self.table_name]
        else:
            NewTable: Table = Table(self.table_name,metadata_obj,extend_existing=True)
            cattr: PomClassAttributes
            for cattr in self.class_attributes:
                PyType = None
                pom_type = cattr.coltype.lower()
                if pom_type == 'varchar':
                    PyType = String(cattr.colsize)
                elif pom_type == 'numeric' and cattr.colprecision == 0 :
                    PyType = Integer
                elif pom_type == 'numeric' and cattr.colprecision > 0:
                    PyType = Float
                else:
                    PyType = String
                print(f"Inferred python type for {cattr.colname}: ",PyType)

                print("Getting super class "+self.super_class)
                PomSuperClass: PomSomMapper = PomSomMapper.get_pom_class(session, self.super_class)
                if PomSuperClass is not None:
                    super_class_table_id = PomSuperClass.table_name+'.id'
                else:
                    print("ERROR coould not find superclass: ",self.super_class)
                    super_class_table_id = 'entities.id'
                if cattr.pkey != 0:
                    NewTable.append_column(Column(cattr.colname,PyType,ForeignKey(super_class_table_id), primary_key=True),replace_existing=True)
                else:
                    NewTable.append_column(Column(cattr.colname,PyType,primary_key=False),replace_existing=True)

            MyORM = Entity.get_orm_for_pom_class(self.id)       
            SuperORM = Entity.get_orm_for_pom_class(self.super_class)
            props = {
                '__table__': NewTable,
                '__mapper_args__':{'polymorphic_identity':self.id}
            }

            newORM = type(self.id.capitalize(),(SuperORM,),props)            
            #print("----")
            #print(repr(NewTable))
            #self.__table__= NewTable

    @classmethod
    def get_pom_class(cls,session, pom_class_name: String):
        """
        Return the pom_class object for a given pom_class_name.

        See also Entity.get_orm_for_pom_class

        """
        pom_class: Optional["PomSomMapper"] = session.query(Entity).get(pom_class_name)
        return pom_class

    def __repr__(self):
        return (
            f'PomSomMapper(id="{self.id}", '
            f'table_name="{self.table_name}", '
            f'class_group="{self.class_group}", '
            f'super_class="{self.super_class}" '
            f')'
        )

    def __str__(self):
        r = f'{self.id} table {self.table_name} super {self.super_class}\n'
        for cattr in self.class_attributes:
            r = r + f'{cattr.the_class}.{cattr.name} \tclass {cattr.colclass} \tcol {cattr.colname} \ttype {cattr.coltype} size {cattr.colsize} precision {cattr.colprecision} primary key {cattr.pkey} \n'
        return r


## Class attributes

    +--------------+--------------+------+-----+---------+-------+
    | Field        | Type         | Null | Key | Default | Extra |
    +--------------+--------------+------+-----+---------+-------+
    | the_class    | varchar(64)  | YES  | MUL | NULL    |       |
    | name         | varchar(32)  | YES  |     | NULL    |       |
    | colname      | varchar(32)  | YES  |     | NULL    |       |
    | colclass     | varchar(32)  | YES  |     | NULL    |       |
    | coltype      | varchar(32)  | YES  |     | NULL    |       |
    | colsize      | decimal(5,0) | YES  |     | NULL    |       |
    | colprecision | decimal(6,0) | YES  |     | NULL    |       |
    | pkey         | decimal(1,0) | YES  |     | NULL    |       |
    +--------------+--------------+------+-----+---------+-------+


In [15]:
class PomClassAttributes(Base):
    __tablename__ = 'class_attributes'

    the_class = Column(String, ForeignKey('classes.id'),primary_key=True)
    
    pom_class = relationship("PomSomMapper",foreign_keys=[the_class], back_populates='class_attributes')
    name = Column(String(32), primary_key=True)
    colname = Column(String(32))
    colclass = Column(String(32))
    coltype = Column(String)
    colsize = Column(Integer)
    colprecision = Column(Integer)
    pkey = Column(Integer)

    def __repr__(self):
        return (
            f'PomClassAttributes(the_class="{self.the_class}", '
            f'name="{self.name}", '
            f'colname="{self.colname}", '
            f'colclass="{self.colclass}", '
            f'coltype="{self.coltype}", '
            f'colsize="{self.colsize}", '
            f'colprecision="{self.colprecision}", '
            f'pkey="{self.pkey}" '
            f')'
        )


PomSomMapper.class_attributes = relationship("PomClassAttributes", back_populates="pom_class")



In [132]:
from timelink.kleio.groups import KGroup

for gn in [(g._name,g) for g in KGroup.all_subclasses()]:
    print(gn)



('kleio', <class 'timelink.kleio.groups.KKleio'>)
('source', <class 'timelink.kleio.groups.KSource'>)
('act', <class 'timelink.kleio.groups.KAct'>)
('person', <class 'timelink.kleio.groups.KPerson'>)
('object', <class 'timelink.kleio.groups.KAbstraction'>)
('object', <class 'timelink.kleio.groups.KObject'>)
('ls', <class 'timelink.kleio.groups.KLs'>)
('atr', <class 'timelink.kleio.groups.KAtr'>)
('attr', <class 'timelink.kleio.groups.KAttribute'>)
('rel', <class 'timelink.kleio.groups.KRelation'>)


## Testing

In [133]:
from sqlalchemy.orm import sessionmaker
Session = sessionmaker(bind=engine)
session = Session()


### Attributes

In [134]:
attribute = session.query(Attribute).where(Entity.pom_class  == 'attribute').first()
print(attribute)
print(repr(attribute))

     ls$residencia/barrosas/16970205/obs=%barrosos. 
Attribute(id=Entity(id="$c51-2-att3-12", pom_class="attribute",inside="$c51-2", the_order=164, the_level=5, the_line=136, groupname="ls", updated=2021-08-21 08:33:00, indexed=1974-04-25 00:00:01,), entity="$c51-2", the_type="residencia", the_value="barrosas", the_date="16970205"", obs=%barrosos. )


In [135]:
entity = attribute.the_entity
print(entity)

pnoivo$domingos andre/m/id=$c51-2


### Relations

In [136]:
relation = session.query(Relation).where(Relation.pom_class  == 'relation').first()

print(repr(relation))
print(relation)


Relation(id=Entity(id="$c51-2-att2-2-geo-rel", pom_class="relation",inside="$c51-2-att2-2-geo", the_order=None, the_level=None, the_line=None, groupname="*georelation", updated=2020-02-26 21:55:45, indexed=1974-04-25 00:00:01,), origin="$c51-2-att2-2-geo", destination="$c51-2", the_type="geografica", the_value="residencia", the_date="16970205"", obs=*automatic relation)
rel$geografica/residencia/domingos andre/$c51-2/16970205  /obs=*automatic relation


In [137]:
relation = session.query(Relation).where(Relation.the_type  == 'parentesco').first()
print(repr(relation))

print(relation.org)
print("   ",relation)
print(relation.dest)

Relation(id=Entity(id="01705-a5e-rel17-4", pom_class="relation",inside="01705-a5e", the_order=274, the_level=5, the_line=331, groupname="rel", updated=2021-11-06 09:21:08, indexed=1974-04-25 00:00:01,), origin="01705-a5e", destination="1705-a5ea", the_type="parentesco", the_value="foi mulher", the_date="17051005"", obs=None)
referida$maria fernandes/f/id=01705-a5e
    rel$parentesco/foi mulher/manuel cordeiro/1705-a5ea/17051005
referido$manuel cordeiro/m/id=1705-a5ea


### Acts

In [138]:
act = session.query(Act).where(Entity.pom_class  == 'act').first()

print(repr(act))
print(act)

Act(id=Entity(id="1700-142", pom_class="act",inside="casamentos 1700", the_order=2, the_level=2, the_line=6, groupname="cas", updated=2020-06-17 18:18:21, indexed=2020-06-17 18:18:21,), the_type="cas", the_date="17000110", local="luis alvares pinto", ref="igreja matriz", obs=None)
cas$1700-142/17000110/type=cas/ref=igreja matriz/loc=luis alvares pinto


### Sources

In [139]:
source = session.query(Source).where(Entity.pom_class  == 'source').first()

print(repr(source))
print(source)

Source(id=Entity(id="1D-8-2-4", pom_class="source",inside="root", the_order=1, the_level=1, the_line=4, groupname="fonte", updated=2021-11-02 11:06:17, indexed=1974-04-25 00:00:01,), the_type="livro de notas", the_date="16910314", local="AUC D-8-2-4", ref="None", kleiofile="/kleio-home/sources/soure-fontes/sources/1685-1720/notariais/1D-8-2-4.cli", replaces="n2.23", obs=legivel)
fonte$1D-8-2-4/16910314/type=livro de notas/ref=None/loc=AUC D-8-2-4/kleiofile=/kleio-home/sources/soure-fontes/sources/1685-1720/notariais/1D-8-2-4.cli/replaces=n2.23  /obs=legivel


### Persons

In [140]:

from sqlalchemy import select

stmt = select(Person).where(Person.pom_class  == 'person').limit(1)
print(stmt)
result = session.execute(stmt)
person = result.scalars().all()[0]
print(person)

SELECT entities.class, persons.id, entities.id AS id_1, entities.inside, entities.the_order, entities.the_level, entities.the_line, entities.groupname, entities.updated, entities.indexed, CASE WHEN (entities.class IN ([POSTCOMPILE_class_1])) THEN entities.class ELSE :param_1 END AS _sa_polymorphic_on, persons.name, persons.sex, persons.obs 
FROM entities JOIN persons ON entities.id = persons.id 
WHERE entities.class = :class_2
 LIMIT :param_2
pnoivo$domingos andre/m/id=$c51-2


In [142]:
from sqlalchemy import select,text

stmt = select(text(' id from classes'))
print(stmt)
result = session.execute(stmt)
ids = result.scalars().all()
print(ids)

SELECT  id from classes
['act', 'acta', 'acusacoes', 'aforamento', 'attribute', 'cartaperdao', 'caso', 'class', 'crime', 'divida', 'entity', 'escritura', 'geoentity', 'geometry', 'good', 'group_element', 'item', 'lgraph', 'object', 'person', 'relation', 'rentity', 'rgeoentity', 'robject', 'rperson', 'source']


### PomSomMapper (table classes)

In [143]:
pom_classes = session.query(PomSomMapper).where(Entity.pom_class  == 'class').all()
for pom_class in pom_classes:
    print(repr(pom_class))
    print(pom_class)

PomSomMapper(id="act", table_name="acts", class_group="historical-act", super_class="entity" )
act table acts
act.date 	class date 	col the_date 	type varchar size 24 precision 0 primary key 0 
act.id 	class id 	col id 	type varchar size 64 precision 0 primary key 1 
act.loc 	class loc 	col loc 	type varchar size 64 precision 0 primary key 0 
act.obs 	class obs 	col obs 	type varchar size 16654 precision 0 primary key 0 
act.ref 	class ref 	col ref 	type varchar size 64 precision 0 primary key 0 
act.type 	class type 	col the_type 	type varchar size 32 precision 0 primary key 0 

PomSomMapper(id="acta", table_name="actas", class_group="amz", super_class="act" )
acta table actas
acta.ano 	class year 	col the_year 	type numeric size 4 precision 0 primary key 0 
acta.dia 	class day 	col the_day 	type numeric size 2 precision 0 primary key 0 
acta.fol 	class fol 	col fol 	type varchar size 64 precision 0 primary key 0 
acta.id 	class id 	col id 	type varchar size 64 precision 0 primary key

In [145]:
for pom_class in [pc for pc in pom_classes if pc.id in ['entity',
'attribute','relation','person','object','source','act',
'rperson','robject','rentity','geoentity','rgeoentity','good']]:
    print(f"'{pom_class.id}': [")
    print(repr(pom_class),',')
    for cattr in pom_class.class_attributes:
        print(repr(cattr),',')
    print('],')

'act': [
PomSomMapper(id="act", table_name="acts", class_group="historical-act", super_class="entity" ) ,
PomClassAttributes(the_class="act", name="date", colname="the_date", colclass="the_date", coltype="varchar", colsize="24", colprecision="0", pkey="0" ) ,
PomClassAttributes(the_class="act", name="id", colname="id", colclass="id", coltype="varchar", colsize="64", colprecision="0", pkey="1" ) ,
PomClassAttributes(the_class="act", name="loc", colname="loc", colclass="loc", coltype="varchar", colsize="64", colprecision="0", pkey="0" ) ,
PomClassAttributes(the_class="act", name="obs", colname="obs", colclass="obs", coltype="varchar", colsize="1024", colprecision="0", pkey="0" ) ,
PomClassAttributes(the_class="act", name="ref", colname="ref", colclass="ref", coltype="varchar", colsize="64", colprecision="0", pkey="0" ) ,
PomClassAttributes(the_class="act", name="type", colname="the_type", colclass="the_type", coltype="varchar", colsize="32", colprecision="0", pkey="0" ) ,
],
'attribute':

## Examining tables

In [477]:
metadata_obj = Base.metadata
for tname in metadata_obj.tables.keys():
    print(tname)
    for col in metadata_obj.tables[tname].columns:
        print("    ",col.name,col.type)


entities
     id VARCHAR
     class VARCHAR
     inside VARCHAR
     the_order INTEGER
     the_level INTEGER
     the_line INTEGER
     groupname VARCHAR
     updated DATETIME
     indexed DATETIME
attributes
     id VARCHAR
     entity VARCHAR
     the_type VARCHAR
     the_value VARCHAR
     the_date VARCHAR
     obs VARCHAR
relations
     id VARCHAR
     origin VARCHAR
     destination VARCHAR
     the_type VARCHAR
     the_value VARCHAR
     the_date VARCHAR
     obs VARCHAR
persons
     id VARCHAR
     name VARCHAR
     sex VARCHAR(1)
     obs VARCHAR
objects
     id VARCHAR
     name VARCHAR
     the_type VARCHAR(32)
     obs VARCHAR
acts
     id VARCHAR
     the_type VARCHAR(32)
     the_date VARCHAR
     loc VARCHAR
     ref VARCHAR
     obs VARCHAR
sources
     id VARCHAR
     the_type VARCHAR(32)
     the_date VARCHAR
     loc VARCHAR
     ref VARCHAR
     kleiofile VARCHAR
     replaces VARCHAR
     obs VARCHAR
classes
     id VARCHAR
     table_name VARCHAR
     group_name

## Dynamic mappings for user defined SomPom classes

In [478]:
from sqlalchemy import Table, Column,Integer, String, Float
pytables = metadata_obj.tables
pom_classes = session.query(PomSomMapper).where(Entity.pom_class  == 'class').all()
pom_class: PomSomMapper
for pom_class in pom_classes:
    if pom_class.table_name in pytables.keys():
        print(pom_class.id,"maps to ")
        print("    ",repr(pytables[pom_class.table_name]))
    else:
        print()
        print(pom_class.id, "python table missing. Creating new")
        print("PomSom information")
        print(pom_class)

        NewTable: Table = Table(pom_class.table_name,metadata_obj,extend_existing=True)

        cattr: PomClassAttributes
        for cattr in pom_class.class_attributes:
            PyType = None
            pom_type = cattr.coltype.lower()
            if pom_type == 'varchar':
                PyType = String(cattr.colsize)
            elif pom_type == 'numeric' and cattr.colprecision == 0 :
                PyType = Integer
            elif pom_type == 'numeric' and cattr.colprecision > 0:
                PyType = Float
            else:
                PyType = String
            print(f"Inferred python type for {cattr.colname}: ",PyType)

            print("Getting super class "+pom_class.super_class)
            PomSuperClass: PomSomMapper = PomSomMapper.get_pom_class(session, pom_class.super_class)
            if PomSuperClass is not None:
                super_class_table_id = PomSuperClass.table_name+'.id'
            else:
                print("ERROR coould not find superclass: ",pom_class.super_class)
                super_class_table_id = 'entities.id'
            if cattr.pkey != 0:
                NewTable.append_column(Column(cattr.colname,PyType,ForeignKey(super_class_table_id), primary_key=True),replace_existing=True)
            else:
                NewTable.append_column(Column(cattr.colname,PyType,primary_key=False),replace_existing=True)
        
        print("----")
        print(repr(NewTable))
        pom_class.__table__= NewTable
    

act maps to 
     Table('acts', MetaData(), Column('id', String(), ForeignKey('entities.id'), table=<acts>, primary_key=True, nullable=False), Column('the_type', String(length=32), table=<acts>), Column('the_date', String(), table=<acts>), Column('loc', String(), table=<acts>), Column('ref', String(), table=<acts>), Column('obs', String(), table=<acts>), schema=None)

acta python table missing. Creating new
PomSom information
acta table actas super act
acta.ano 	class year 	col the_year 	type numeric size 4 precision 0 primary key 0 
acta.dia 	class day 	col the_day 	type numeric size 2 precision 0 primary key 0 
acta.fol 	class fol 	col fol 	type varchar size 64 precision 0 primary key 0 
acta.id 	class id 	col id 	type varchar size 64 precision 0 primary key 1 
acta.mes 	class month 	col the_month 	type numeric size 2 precision 0 primary key 0 
acta.obs 	class obs 	col obs 	type varchar size 16654 precision 0 primary key 0 
acta.resumo 	class resumo 	col resumo 	type varchar size 102

### Metadata with new dynamic tables

In [479]:
newtables = metadata_obj.tables
for name, table in newtables.items():
    print(name)
    for col in table.columns:
        print("    ",col.name,col.type)

entities
     id VARCHAR
     class VARCHAR
     inside VARCHAR
     the_order INTEGER
     the_level INTEGER
     the_line INTEGER
     groupname VARCHAR
     updated DATETIME
     indexed DATETIME
attributes
     id VARCHAR
     entity VARCHAR
     the_type VARCHAR
     the_value VARCHAR
     the_date VARCHAR
     obs VARCHAR
relations
     id VARCHAR
     origin VARCHAR
     destination VARCHAR
     the_type VARCHAR
     the_value VARCHAR
     the_date VARCHAR
     obs VARCHAR
persons
     id VARCHAR
     name VARCHAR
     sex VARCHAR(1)
     obs VARCHAR
objects
     id VARCHAR
     name VARCHAR
     the_type VARCHAR(32)
     obs VARCHAR
acts
     id VARCHAR
     the_type VARCHAR(32)
     the_date VARCHAR
     loc VARCHAR
     ref VARCHAR
     obs VARCHAR
sources
     id VARCHAR
     the_type VARCHAR(32)
     the_date VARCHAR
     loc VARCHAR
     ref VARCHAR
     kleiofile VARCHAR
     replaces VARCHAR
     obs VARCHAR
classes
     id VARCHAR
     table_name VARCHAR
     group_name

## Creating new Mapped Classes for the inferred tables

        from sqlalchemy.orm import registry

        mapper_registry = registry()

        my_table = Table(
            "my_table",
            mapper_registry.metadata,
            Column('id', Integer, primary_key=True)
        )

        class MyClass:
            pass

        mapper_registry.map_imperatively(MyClass, my_table)


https://docs.sqlalchemy.org/en/14/orm/mapping_api.html

In [480]:
casos_table = metadata_obj.tables['casos']
print(casos_table.name)
for col in casos_table.columns:
    print("    ",col.name,col.type)
s = select(casos_table)
print(s)
results = session.execute(s)
for c in results:
    print(c)

casos
     id VARCHAR(64)
     obs VARCHAR(16654)
     the_type VARCHAR(32)
SELECT casos.id, casos.obs, casos.the_type 
FROM casos
('1708-c1', 'na pronuncia sao setenciados a fazerem vida marital e ele a a tratar bem. pronuncia: admoestacao,resultado: fez termo', 'violf')
('1708-c2', None, 'nfvida')
('1708-c3', None, 'amanc')
('1708-c4', None, 'amanc')
('1708-c5', None, 'cons')
('1708-c6', None, 'ma lin')
('1708-c7', None, 'violf')
('1708-c8', None, 'ma lin')
('1708-c9', " ela casou-se em 21 de Fevereiro de 1707, ha' um ano e meio", 'amanc')
('c1690-luisa-manrique-maria-simoes', '"""\n                   \n                  Ela vive longe do marido, numa casa por conta dele. \n                  Já acusados em visitas anteriores44relacao com muitos anos.\n\n                  """', 'amanc')
('c1692-ana-simoes-feiticeira', 'ana simoes,2114a', 'feitic')
('c1692-antonio-catarina-fernandes', 'tiveram filho(a). 2.o a t.a 23 manuel goncalves, ele foi prezo por ordem do pai dela.', 'amanc')
('c1

In [481]:
print(repr(casos_table))

Table('casos', MetaData(), Column('id', String(length=Decimal('64')), ForeignKey('objects.id'), table=<casos>, primary_key=True, nullable=False), Column('obs', String(length=Decimal('16654')), table=<casos>), Column('the_type', String(length=Decimal('32')), table=<casos>), schema=None)


In [482]:
#see https://www.mail-archive.com/sqlalchemy@googlegroups.com/msg38030.html

pom_class_name = 'caso'
pom_class = session.query(PomSomMapper).get(pom_class_name)
pom_super_class = session.execute(select(PomSomMapper).where(PomSomMapper.id == pom_class.super_class)).scalars().one_or_none()
print(pom_super_class)

SuperORM = Entity.get_orm_for_pom_class(pom_super_class.id)

print(SuperORM)

props = {
    '__table__': casos_table,
    '__mapper_args__':{'polymorphic_identity':pom_class.id}
}

newORM = type(pom_class_name.capitalize(),(SuperORM,),props)
#Entity.registry.map_imperatively(newORM, casos_table,inherits=Object,polymorphic_identity=pom_class.id)
s = select(newORM)
print(s)
results = session.execute(s)
for c in results:
    print(c)


    

object table objects super entity
object.id 	class id 	col id 	type varchar size 64 precision 0 primary key 1 
object.name 	class name 	col name 	type varchar size 64 precision 0 primary key 0 
object.obs 	class obs 	col obs 	type varchar size 16654 precision 0 primary key 0 
object.type 	class type 	col the_type 	type varchar size 32 precision 0 primary key 0 

<class '__main__.Object'>
SELECT entities.class, casos.id, objects.id AS id_1, entities.id AS id_2, entities.inside, entities.the_order, entities.the_level, entities.the_line, entities.groupname, entities.updated, entities.indexed, CASE WHEN (entities.class IN ([POSTCOMPILE_class_1])) THEN entities.class ELSE :param_1 END AS _sa_polymorphic_on, objects.name, casos.the_type, objects.the_type AS the_type_1, casos.obs, objects.obs AS obs_1 
FROM entities JOIN objects ON entities.id = objects.id JOIN casos ON objects.id = casos.id
(Object(id=Entity(id="1708-c1", pom_class="caso",inside="1708-t1", the_order=9, the_level=4, the_line=

/var/folders/sx/x6pyh9097n98mlz29_6njdj00000gn/T/ipykernel_63215/1205763381.py:17: SAWarning: Implicitly combining column objects.obs with column casos.obs under attribute 'obs'.  Please configure one or more attributes for these same-named columns explicitly.
  newORM = type(pom_class_name.capitalize(),(SuperORM,),props)
/var/folders/sx/x6pyh9097n98mlz29_6njdj00000gn/T/ipykernel_63215/1205763381.py:17: SAWarning: Implicitly combining column objects.the_type with column casos.the_type under attribute 'the_type'.  Please configure one or more attributes for these same-named columns explicitly.
  newORM = type(pom_class_name.capitalize(),(SuperORM,),props)


In [483]:
newORM.__mapper_args__

{'polymorphic_identity': 'caso'}

In [484]:
for mapper in Entity.registry.mappers:
    print(f' Table {mapper.local_table} is mapped to {mapper.class_} through entities.class={mapper.polymorphic_identity}')


 Table persons is mapped to <class '__main__.Person'> through entities.class=person
 Table class_attributes is mapped to <class '__main__.PomClassAttributes'> through entities.class=None
 Table entities is mapped to <class '__main__.Entity'> through entities.class=entity
 Table objects is mapped to <class '__main__.Object'> through entities.class=object
 Table classes is mapped to <class '__main__.PomSomMapper'> through entities.class=class
 Table relations is mapped to <class '__main__.Relation'> through entities.class=relation
 Table attributes is mapped to <class '__main__.Attribute'> through entities.class=attribute
 Table casos is mapped to <class '__main__.Caso'> through entities.class=caso
 Table sources is mapped to <class '__main__.Source'> through entities.class=source
 Table acts is mapped to <class '__main__.Act'> through entities.class=act


In [485]:
print(Entity.mapped_pom_classes())

['attribute', 'relation', 'person', 'object', 'act', 'source', 'class']


In [492]:
print("Dictionnary for mapping table to ORM classes")
table_to_orm= {subclass.__mapper__.local_table.name:subclass for subclass in Entity.get_subclasses()}
print(table_to_orm)
print("Dictionnary for mapping pom_class to ORM classes")
pom_class_to_orm= {subclass.__mapper__.polymorphic_identity:subclass for subclass in Entity.get_subclasses()}
print(pom_class_to_orm)



Dictionnary for mapping table to ORM classes
{'attributes': <class '__main__.Attribute'>, 'relations': <class '__main__.Relation'>, 'persons': <class '__main__.Person'>, 'casos': <class '__main__.Caso'>, 'objects': <class '__main__.Object'>, 'acts': <class '__main__.Act'>, 'sources': <class '__main__.Source'>, 'classes': <class '__main__.PomSomMapper'>}
Dictionnary for mapping pom_class to ORM classes
{'attribute': <class '__main__.Attribute'>, 'relation': <class '__main__.Relation'>, 'person': <class '__main__.Person'>, 'caso': <class '__main__.Caso'>, 'object': <class '__main__.Object'>, 'act': <class '__main__.Act'>, 'source': <class '__main__.Source'>, 'class': <class '__main__.PomSomMapper'>}


In [491]:
Caso.__mapper__.local_table.name


'objects'

### Next steps

* Entity new methods
    * get_orm_for_table(table): return table_to_orm.get(table,None)
    * get_orm_for_pom_class(pom_class) return pom_class_to_orm.get(pom_class,None) 
    
* PomToSom generate table done above refactor as method
    * newtable= blah blah
    * if id=entity exit
    * BaseForThis = Entity.get_orm_for_pom_class(self.super_class)
    * Use Type to create empty class extending BaseForthis into NewORM
    * Entity.registry.map_imperatively(NewORM, newTable)
    * set the polymorphic_identity of newTable = self.id HOW? just set __mapper_args__?


We can check if the new table needs to be created

    # The recommended way to check for existence
    sqlalchemy.inspect(engine).has_table("BOOKS")

In [72]:
result=session.query(PomSomMapper.id,PomSomMapper.super_class).all()
print(result)


[('act', 'entity'), ('acta', 'act'), ('acusacoes', 'object'), ('aforamento', 'good'), ('attribute', 'entity'), ('cartaperdao', 'act'), ('caso', 'object'), ('class', 'entity'), ('crime', 'object'), ('divida', 'object'), ('entity', 'root'), ('escritura', 'act'), ('geoentity', 'entity'), ('good', 'object'), ('group_element', 'entity'), ('item', 'object'), ('lgraph', 'entity'), ('object', 'entity'), ('person', 'entity'), ('relation', 'entity'), ('rgeoentity', 'rentity'), ('robject', 'rentity'), ('rperson', 'rentity'), ('source', 'entity')]


## Testing handling of missing mappings

In [24]:
C = PomSomMapper.find_pom_class(session, 'acta')
print(C)

act


In [25]:
acta = session.query(Act).where(Entity.pom_class  == 'acta').first()

print(repr(acta))
print(acta)

Act(id=Entity(id="amz1", pom_class="acta",inside="mis-mesa-1", the_order=2, the_level=2, the_line=6, groupname="amz", updated=2021-11-01 04:22:41, indexed=2021-11-01 04:22:41,), the_type="amz", the_date="16831003", local="None", ref="None", obs=None)
amz$amz1/16831003/type=amz/ref=None/loc=None


In [365]:
classes_from_entities = session.query(Entity.pom_class.distinct()).all()
print(classes_from_entities)


[('act',), ('acta',), ('acusacoes',), ('aforamento',), ('aregister',), ('attribute',), ('cartaperdao',), ('caso',), ('class',), ('crime',), ('divida',), ('escritura',), ('geoentity',), ('good',), ('group_element',), ('item',), ('object',), ('person',), ('relation',), ('rperson',), ('source',)]


In [326]:
print(Entity.__subclasses__())
for aclass in Entity.__subclasses__():
    print(aclass.__tablename__)

[<class '__main__.Attribute'>, <class '__main__.Relation'>, <class '__main__.Person'>, <class '__main__.Object'>, <class '__main__.Act'>, <class '__main__.Source'>, <class '__main__.PomClass'>]
attributes
relations
persons
objects
acts
sources
classes


In [63]:
from sqlalchemy import select
from sqlalchemy.orm import Session

stmt = select(Person).first()
print(stmt)
with Session(engine) as session:
    for row in session.execute(stmt):
        entity: Entity=row[0]
        print(f'{entity.groupname}${entity.name}/id={entity.id}#{entity.pom_class}')
        for attribute in entity.attributes:
            print(f'     ls${attribute.the_type}/{attribute.the_value}/{attribute.the_date}',end='')
            if attribute.obs is not None:
                print(f'/obs={attribute.obs}')
            else:
                print()
        if len(entity.rels_out)>0:
            for rel in entity.rels_out:
                print(f'>{rel}')
        if len(entity.rels_in)>0:
            for rel in entity.rels_in:
                    print(rel,'<')               


SELECT entities.class, persons.id, entities.id AS id_1, entities.inside, entities.the_order, entities.the_level, entities.the_line, entities.groupname, entities.updated, entities.indexed, persons.name, persons.sex, persons.obs 
FROM entities JOIN persons ON entities.id = persons.id 
WHERE persons.id = :id_2
n$Arnaldo Henriques de Abreu/id=140625#person
     ls$código-de-referência/"PT/AUC/ELU/UC-AUC/B/001-001/A/000336"/20200211
     ls$data-do-registo/2018-04-19 14:33:58/20200211
     ls$url/"https://pesquisa.auc.uc.pt/details?id=140625"/20200211
     ls$uc.entrada/1835-10-17/1835-10-17
     ls$uc.saida/1838-10-09/1838-10-09
     ls$uc.entrada.ano/1835/1835-10-17
     ls$uc.saida.ano/1838/1838-10-09
     ls$nome/Arnaldo Henriques de Abreu/1835-10-17
     ls$nome.primeiro/Arnaldo/1835-10-17
     ls$nome.apelido/Henriques de Abreu/1835-10-17
     ls$nome.apelido/Abreu/1835-10-17
     ls$naturalidade/Porto/1835-10-17
     ls$naturalidade.ano/Porto.1835/1835-10-17
     ls$nome-geografico/P

## Source files

In [37]:
from pathlib import Path

kleio_files = [f.stem for f in list(Path('../sources').rglob('*.cli'))]
print("Number of kleio_files:", len(kleio_files))

imported_sources = %sql select sources.id, sources.kleiofile, entities.updated from sources left join entities on (sources.id=entities.id) order by updated desc 
sources_in_db = [s.id for s in imported_sources]
print("Number of imported files:",len(sources_in_db))

print("Files not in the database:", set(kleio_files)-set(sources_in_db))
print("Imported sources no file found:", set(sources_in_db)-set(kleio_files))



Number of kleio_files: 235
235 rows affected.
Number of imported files: 235
Files not in the database: set()
Imported sources no file found: set()


## Analyse attributes extracted from records

### Attributes in the database

In [8]:
nml = %sql select the_type, count(*) as tot from attributes group by the_type
for the_type, tot in nml:
    print(f'{tot:6} | {the_type}')

28 rows affected.
105300 | código-de-referência
   275 | colegio
105300 | data-do-registo
   186 | ec
 53627 | exame
 94164 | faculdade
 94164 | faculdade.ano
106372 | grau
106372 | grau.ano
319291 | matricula
319291 | matricula.ano
 98904 | naturalidade
 98904 | naturalidade.ano
105300 | nome
107486 | nome-geografico
101392 | nome-geografico.ano
198660 | nome.apelido
105300 | nome.primeiro
  8155 | nome.vide
  9975 | nota
  3142 | ordem-religiosa
  7036 | padre
  4170 | titulo
105300 | uc.entrada
 99014 | uc.entrada.ano
105300 | uc.saida
 99281 | uc.saida.ano
105300 | url


In [11]:
p1540 = %sql select id, name, pobs from nattributes where the_type='uc.entrada' and the_value like '1540%'
for id, name, pobs in p1540:
    print(f'{id:5} | {name} \n {pobs}\n\n')

473 rows affected.


ValueError: Unknown format code 'r' for object of type 'str'